In [1]:
!pip install --upgrade --user --quiet google-cloud-aiplatform
!pip install --quiet transformers datasets
!pip install --quiet python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\souma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\souma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\souma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [4]:
import os

import vertexai

from dotenv import load_dotenv

load_dotenv()
PROJECT_ID = os.getenv("PROJECT_ID")
 
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
import IPython.display
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

In [6]:
MODEL_ID = "gemini-1.5-flash-002"  # @param {type:"string"}

model = GenerativeModel(MODEL_ID)

In [7]:
generation_config = GenerationConfig(
    temperature=0,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
}

example_model = GenerativeModel(
    MODEL_ID,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [8]:
import pickle

def save_variables(filename="variables.pkl"):
    global_vars = {k: v for k, v in globals().items() 
                   if not k.startswith("__") and k != "save_variables"}
    safe_vars = {}
    for k, v in global_vars.items():
        try:
            pickle.dumps(v)  
            safe_vars[k] = v  
        except (pickle.PickleError, AttributeError, TypeError):
            print(f"⚠️ Skipping non-pickleable variable: {k}")

    with open(filename, "wb") as f:
        pickle.dump(safe_vars, f)

    print(f"✅ Variables saved to {filename}")
save_variables()


⚠️ Skipping non-pickleable variable: get_ipython
⚠️ Skipping non-pickleable variable: exit
⚠️ Skipping non-pickleable variable: quit
⚠️ Skipping non-pickleable variable: open
⚠️ Skipping non-pickleable variable: sys
⚠️ Skipping non-pickleable variable: os
⚠️ Skipping non-pickleable variable: vertexai
⚠️ Skipping non-pickleable variable: IPython
⚠️ Skipping non-pickleable variable: pickle
✅ Variables saved to variables.pkl


In [9]:
import json
import re

with open("input-text.js", "r", encoding="utf-8") as file:
    content = file.read()

match = re.search(r'export const inputData = (.*);', content, re.DOTALL)

if match:
    extracted_data = match.group(1).strip()
    # print("Extracted Data Before Parsing:", extracted_data)
else:
    raise ValueError("Invalid input-text.js format - No match found!")

response = example_model.generate_content([extracted_data])

# Escape $ and backticks ` by adding a backslash before them
escaped_text = re.sub(r'([$`])', r'\\\1', response.text)

output_data = f"export const outputData = {{ text : `{escaped_text}` }};\n"

with open("output-text.js", "w", encoding="utf-8") as file:
    file.write(output_data)

# print("Processed output written to output-text.js")


Extracted Data Before Parsing: ("Need a website that makes blog")
Processed output written to output.js
